<a href="https://colab.research.google.com/github/ravibalebilalu/ravibalebilalu/blob/main/pytorch_sun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
bank = pd.read_csv("Churn_Modelling.csv")
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
bank['Geography'] = le.fit_transform(bank['Geography'])
bank['Gender'] = le.fit_transform(bank['Gender'])
bank = bank.drop(['RowNumber','CustomerId','Surname'],axis = "columns")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = bank.drop(['Exited'],axis = 'columns').values
y = bank['Exited'].values
x = sc.fit_transform(x)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state = 1)
x_train.shape



 

In [ ]:
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

class ANN_model(nn.Module):
  def __init__(self,input_features = x_train.shape[1],hidden1 = 100,hidden2 = 100,output = 2):
    super().__init__()
    self.f_connected1 = nn.Linear(input_features,hidden1)
    self.f_connected2 = nn.Linear(hidden1,hidden2)
    self.out = nn.Linear(hidden2,output)
  def forward(self,x):
    x = F.relu(self.f_connected1(x))
    x = F.relu(self.f_connected2(x))
    x = self.out(x)
    return x

torch.manual_seed(20)
model = ANN_model()
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

epochs = 1000
final_loss = []
for i in range(epochs):
  i+=1
  y_pred = model.forward(x_train)
  loss = loss_function(y_pred,y_train)
  final_loss.append(loss)
  if i%10 == 1:
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
final_loss = torch.tensor(final_loss,requires_grad = True).detach().numpy()


plt.plot(range(epochs),final_loss)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()


In [ ]:

predictions = []
with torch.no_grad():
  for i , data in enumerate(x_test):
    y_pred = model(data)
    predictions.append(y_pred.argmax().item())
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,predictions))



In [ ]:
torch.save(model,"bank_churn.pt")

In [ ]:
bank_c = torch.load("bank_churn.pt")
bank_c.eval()